# PetFinder Pawpularity: Plan

Objectives:
- Win a medal (RMSE ≤ 17.097).
- Build a strong, GPU-accelerated image model with robust CV and fast iteration.

Milestones:
1) Environment + Data Check
   - Verify GPU availability, install PyTorch/torchvision, check image counts and CSVs.
   - Lock folds (StratifiedKFold on Pawpularity bins). Save folds.

2) Baseline
   - Simple image-only CNN (timm pretrained, e.g., convnext_tiny or efficientnet_v2_s).
   - Input size 384, light augs, MSE loss; predict Pawpularity scaled to 0-1 then rescale.
   - 5-fold CV, early stopping, mixed precision, EMA, cosine LR.

3) Add Metadata
   - Use train.csv binary attributes (Subject Focus, Eyes, etc.).
   - Tabular head + image backbone (late fusion).

4) Improve
   - Resolution sweep (384→512), TTA, better augs, CutMix/TrivialAugment optional.
   - Seeds x2, model variants, simple weighted blend.

5) Error Analysis
   - OOF diagnostics, bins by target, per-fold checks.

6) Submission
   - Inference with TTA, generate submission.csv.

Validation:
- 5-fold Stratified by binned Pawpularity (e.g., 10 bins), deterministic seeds.
- All transforms fitted per-fold. Cache OOF/test preds.

Logging/Speed:
- Print progress/time per epoch/fold, early stop patience, checkpoint best RMSE.
- Subsample for smoke tests before full runs.

Next:
- Run env check cell: GPU, package install, data sanity; then lock folds and request expert review.

In [3]:
# Environment and data sanity check
import sys, subprocess, time, os, glob, platform, json
print(f"Python: {sys.version.split()[0]} | Platform: {platform.platform()}")

def pip_run(args):
    print(f"\n[pip] {' '.join(args)}", flush=True)
    cmd = [sys.executable, '-m', 'pip'] + args
    subprocess.run(cmd, check=True)

# 1) Install GPU builds of torch/torchvision for CUDA 12.1 (pin to avoid surprises)
pip_run(['install', '--index-url', 'https://download.pytorch.org/whl/cu121', 'torch==2.5.1+cu121', 'torchvision==0.20.1+cu121'])

# 2) Install timm without deps to avoid touching torch/torchvision
pip_run(['install', 'timm', '--no-deps', '--upgrade-strategy', 'only-if-needed'])

# 3) Install remaining deps without upgrading torch
other_pkgs = ['albumentations>=1.4.0', 'opencv-python-headless', 'pandas', 'numpy', 'scikit-learn', 'tqdm', 'matplotlib']
pip_run(['install', '--upgrade-strategy', 'only-if-needed'] + other_pkgs)

import torch, torchvision
import pandas as pd, numpy as np

print(f"Torch: {torch.__version__} | TorchVision: {torchvision.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    print(f"GPU Memory: {props.total_memory/1024**3:.1f} GB")

# Data sanity
train_csv = 'train.csv'; test_csv = 'test.csv'
train_dir = 'train'; test_dir = 'test'
assert os.path.exists(train_csv) and os.path.exists(test_csv), 'CSV files missing'
assert os.path.isdir(train_dir) and os.path.isdir(test_dir), 'Image dirs missing'
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
print('train.csv shape:', train_df.shape, '| test.csv shape:', test_df.shape)
print('Columns (train):', list(train_df.columns))
print('Columns (test):', list(test_df.columns))

# Check image counts and missing files
train_imgs = set(os.path.splitext(os.path.basename(p))[0] for p in glob.glob(os.path.join(train_dir, '*.jpg')))
test_imgs = set(os.path.splitext(os.path.basename(p))[0] for p in glob.glob(os.path.join(test_dir, '*.jpg')))
missing_train = [i for i in train_df['Id'].astype(str).tolist() if i not in train_imgs]
missing_test = [i for i in test_df['Id'].astype(str).tolist() if i not in test_imgs]
print(f"Train images on disk: {len(train_imgs)} | in CSV: {len(train_df)} | missing: {len(missing_train)}")
print(f"Test images on disk: {len(test_imgs)} | in CSV: {len(test_df)} | missing: {len(missing_test)}")
if missing_train[:5]: print('Sample missing train ids:', missing_train[:5])
if missing_test[:5]: print('Sample missing test ids:', missing_test[:5])

# Target distribution quick stats
if 'Pawpularity' in train_df.columns:
    y = train_df['Pawpularity'].values
    print('Pawpularity stats:', {'min': int(np.min(y)), 'max': int(np.max(y)), 'mean': float(np.mean(y)), 'std': float(np.std(y))})
    # propose bins for stratification (20 bins using floor/5 capped to 19)
    bins = np.clip((y // 5).astype(int), 0, 19)
    uniq, counts = np.unique(bins, return_counts=True)
    print('Strat bins (floor/5) distribution sample:', dict(zip(uniq.tolist()[:10], counts.tolist()[:10])))

print('\nEnv & data check complete.')

Python: 3.11.0rc1 | Platform: Linux-6.8.0-1031-azure-x86_64-with-glibc2.35

[pip] install --index-url https://download.pytorch.org/whl/cu121 torch==2.5.1+cu121 torchvision==0.20.1+cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 566.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 371.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 549.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 559.6 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 165.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 271.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 393.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.20 requires huggingface_hub, which is not installed.
timm 1.0.20 requires safetensors, which is not installed.


Torch: 2.5.1+cu121 | TorchVision: 0.20.1+cu121
GPU Available: False
GPU Count: 0
train.csv shape: (8920, 14) | test.csv shape: (992, 13)
Columns (train): ['Id', 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'Pawpularity']
Columns (test): ['Id', 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
Train images on disk: 8920 | in CSV: 8920 | missing: 0
Test images on disk: 992 | in CSV: 992 | missing: 0
Pawpularity stats: {'min': 1, 'max': 100, 'mean': 38.024775784753366, 'std': 20.646554573509412}
Strat bins (floor/5) distribution sample: {0: 184, 1: 147, 2: 267, 3: 528, 4: 1009, 5: 1369, 6: 1256, 7: 974, 8: 746, 9: 535}

Env & data check complete.


/app/.pip-target/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# Create and save stratified 5-fold splits (20 bins)
import os, time
import pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold

t0 = time.time()
train_df = pd.read_csv('train.csv')
assert 'Id' in train_df.columns and 'Pawpularity' in train_df.columns, 'Missing Id/Pawpularity'

# 20-bin stratification via floor/5 capped to 19
y = train_df['Pawpularity'].values.astype(int)
bins = np.clip((y // 5).astype(int), 0, 19)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)
folds = np.full(len(train_df), -1, dtype=int)
for f, (_, val_idx) in enumerate(skf.split(np.zeros(len(bins)), bins)):
    folds[val_idx] = f

train_df['fold'] = folds
assert (train_df['fold'] >= 0).all(), 'Unassigned fold discovered'

# Save folds mapping for reuse
folds_path = 'folds.csv'
train_df[['Id', 'fold']].to_csv(folds_path, index=False)

# Summary
print('Saved:', folds_path, '| shape:', train_df.shape)
print('Fold distribution (counts):', train_df['fold'].value_counts().sort_index().to_dict())
print('Bins per fold (head):')
print(train_df.groupby('fold')['Pawpularity'].agg(['mean','std','min','max']).round(3))
print(f'Done in {time.time()-t0:.2f}s')

Saved: folds.csv | shape: (8920, 15)
Fold distribution (counts): {0: 1784, 1: 1784, 2: 1784, 3: 1784, 4: 1784}
Bins per fold (head):
        mean     std  min  max
fold                          
0     38.031  20.561    1  100
1     37.999  20.636    2  100
2     38.006  20.725    2  100
3     38.066  20.667    1  100
4     38.022  20.671    2  100
Done in 0.02s


In [5]:
# GPU diagnostics
import os, subprocess, sys, torch

def run_cmd(cmd):
    print(f"\n$ {' '.join(cmd)}", flush=True)
    try:
        out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, check=False)
        print(out.stdout)
    except Exception as e:
        print(f"FAILED: {e}")

print('Env vars:')
for k in ['CUDA_VISIBLE_DEVICES','NVIDIA_VISIBLE_DEVICES','NVIDIA_DRIVER_CAPABILITIES']:
    print(f"  {k} = {os.getenv(k)}")

print('\nPyTorch CUDA:')
print('  torch_ver =', torch.__version__)
print('  torch.version.cuda =', torch.version.cuda)
print('  is_available =', torch.cuda.is_available())
print('  device_count =', torch.cuda.device_count())

run_cmd(['nvidia-smi'])
run_cmd(['bash','-lc','ls -l /dev/nvidia*'])
run_cmd(['bash','-lc','cat /proc/driver/nvidia/version || true'])
run_cmd(['bash','-lc','ldconfig -p | grep nvidia || true'])

print('\nDiagnostics complete.')

Env vars:
  CUDA_VISIBLE_DEVICES = None
  NVIDIA_VISIBLE_DEVICES = None
  NVIDIA_DRIVER_CAPABILITIES = None

PyTorch CUDA:
  torch_ver = 2.5.1+cu121
  torch.version.cuda = 12.1
  is_available = False
  device_count = 0

$ nvidia-smi


Failed to initialize NVML: Unknown Error


$ bash -lc ls -l /dev/nvidia*


crw-rw-rw- 1 root root 234,   0 Sep 22 04:48 /dev/nvidia-uvm
crw-rw-rw- 1 root root 234,   1 Sep 22 04:48 /dev/nvidia-uvm-tools
crw-rw-rw- 1 root root 195,   0 Sep 22 04:47 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 Sep 22 04:47 /dev/nvidiactl


$ bash -lc cat /proc/driver/nvidia/version || true


NVRM version: NVIDIA UNIX x86_64 Kernel Module  550.144.06  Mon Apr 14 05:50:23 UTC 2025
GCC version:  gcc version 11.4.0 (Ubuntu 11.4.0-1ubuntu1~22.04.2) 


$ bash -lc ldconfig -p | grep nvidia || true


	libnvidia-ptxjitcompiler.so.1 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-ptxjitcompiler.so.1
	libnvidia-pkcs11.so.550.144.06 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-pkcs11.so.550.144.06
	libnvidia-pkcs11-openssl3.so.550.144.06 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-pkcs11-openssl3.so.550.144.06
	libnvidia-opencl.so.1 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-opencl.so.1
	libnvidia-nvvm.so.4 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-nvvm.so.4
	libnvidia-ml.so.1 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-ml.so.1
	libnvidia-gpucomp.so.550.144.06 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-gpucomp.so.550.144.06
	libnvidia-cfg.so.1 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-cfg.so.1
	libnvidia-allocator.so.1 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libnvidia-allocator.so.1


Diagnostics complete.


/app/.pip-target/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
# CPU fallback: install extras for embeddings + boosted trees
import sys, subprocess
def pip_run(args):
    print(f"[pip] {' '.join(args)}", flush=True)
    subprocess.run([sys.executable, '-m', 'pip'] + args, check=True)

pkgs1 = ['install', '--upgrade-strategy', 'only-if-needed', 'huggingface_hub', 'safetensors']
pkgs2 = ['install', '--upgrade-strategy', 'only-if-needed', 'lightgbm', 'xgboost']
pip_run(pkgs1)
pip_run(pkgs2)
print('Installed huggingface_hub, safetensors, lightgbm, xgboost')

[pip] install --upgrade-strategy only-if-needed huggingface_hub safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 KB 21.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 KB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 397.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 418.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 435.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 422.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 KB 417.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 KB 264.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 450.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 477.8 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 440.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 18.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━╸                 170.9/295.9 MB 22.5 MB/s eta 0:00:06


ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/lib/python3/dist-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/http/client.py", line 465, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ssl.py", line 1278, in recv_into
    return self.read(nbytes, buffer)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ssl.py", line 1134, in read
    return self._sslobj.read(len, buffer)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TimeoutError: The read operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent c

CalledProcessError: Command '['/usr/bin/python3.11', '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed', 'lightgbm', 'xgboost']' returned non-zero exit status 2.

In [7]:
# CPU fallback: extract 224px frozen embeddings for two timm models
import os, time, gc, sys, math, glob
import numpy as np, pandas as pd
from PIL import Image
import torch
import timm
from torch.utils.data import Dataset, DataLoader
from timm.data import resolve_data_config, create_transform

torch.set_num_threads(8)

class ImageDataset(Dataset):
    def __init__(self, ids, folder, transform):
        self.ids = ids
        self.folder = folder
        self.transform = transform
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        img_id = self.ids[idx]
        path = os.path.join(self.folder, f"{img_id}.jpg")
        img = Image.open(path).convert('RGB')
        img = self.transform(img)
        return img

def extract_embeddings(model_name, img_size=224, batch_size=128, num_workers=8):
    t0 = time.time()
    print(f"\n[Emb] {model_name} @ {img_size}")
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    train_ids = train_df['Id'].tolist()
    test_ids = test_df['Id'].tolist()

    # Model
    model = timm.create_model(model_name, pretrained=True, num_classes=0, global_pool='avg')
    model.eval()
    device = torch.device('cpu')
    model.to(device)

    # Transforms based on timm config
    cfg = resolve_data_config({}, model=model)
    cfg['input_size'] = (3, img_size, img_size)
    transform = create_transform(**cfg, is_training=False)

    # Datasets/DataLoaders
    train_ds = ImageDataset(train_ids, 'train', transform)
    test_ds = ImageDataset(test_ids, 'test', transform)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False)

    # Infer embedding dimension
    with torch.no_grad():
        sample = next(iter(train_loader))[:1].to(device)
        emb_dim = model(sample).shape[1]
    print(f"Embedding dim: {emb_dim}")

    def run_loader(loader, n_items):
        X = np.zeros((n_items, emb_dim), dtype=np.float32)
        i0 = 0
        with torch.no_grad():
            for i, xb in enumerate(loader):
                xb = xb.to(device)
                feats = model(xb).cpu().numpy().astype(np.float32)
                X[i0:i0+feats.shape[0]] = feats
                i0 += feats.shape[0]
                if (i+1) % 20 == 0:
                    print(f"  Batches {i+1}/{math.ceil(n_items/loader.batch_size)} | rows {i0}/{n_items} | elapsed {time.time()-t0:.1f}s", flush=True)
        return X

    X_tr = run_loader(train_loader, len(train_ds))
    X_te = run_loader(test_loader, len(test_ds))

    tr_out = f"X_img_train_{model_name.replace('/', '_')}_{img_size}.npy"
    te_out = f"X_img_test_{model_name.replace('/', '_')}_{img_size}.npy"
    np.save(tr_out, X_tr)
    np.save(te_out, X_te)
    print(f"Saved {tr_out} {X_tr.shape}, {te_out} {X_te.shape} | time {time.time()-t0:.1f}s")
    del model, X_tr, X_te, train_loader, test_loader, train_ds, test_ds
    gc.collect()

models = [
    ('tf_efficientnet_b0', 224),
    ('swin_tiny_patch4_window7_224', 224),
]

for name, sz in models:
    extract_embeddings(name, img_size=sz, batch_size=128, num_workers=8)
print('Embeddings extraction complete.')

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[Emb] tf_efficientnet_b0 @ 224


Embedding dim: 1280


  Batches 20/70 | rows 2560/8920 | elapsed 19.2s


  Batches 40/70 | rows 5120/8920 | elapsed 34.0s


...

Embedding dim: 768


  Batches 20/70 | rows 2560/8920 | elapsed 54.0s


  Batches 40/70 | rows 5120/8920 | elapsed 105.1s


  Batches 60/70 | rows 7680/8920 | elapsed 155.9s


Saved X_img_train_swin_tiny_patch4_window7_224_224.npy (8920, 768), X_img_test_swin_tiny_patch4_window7_224_224.npy (992, 768) | time 201.3s
Embeddings extraction complete.


In [8]:
# CPU fallback: build metadata + simple image stats and save aligned arrays
import os, time, cv2, numpy as np, pandas as pd

def compute_img_stats(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        return [0,0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0]
    h, w = img.shape[:2]
    aspect = (w / max(h,1)) if h>0 else 0.0
    # per-channel means/std in BGR
    ch_means = img.reshape(-1,3).mean(axis=0).tolist()
    ch_stds = img.reshape(-1,3).std(axis=0).tolist()
    # blur metric: Laplacian variance (grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    lap_var = float(lap.var())
    return [w, h, aspect] + ch_means + ch_stds + [lap_var]

t0 = time.time()
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

meta_cols = ['Subject Focus','Eyes','Face','Near','Action','Accessory','Group','Collage','Human','Occlusion','Info','Blur']
assert all(c in train_df.columns for c in meta_cols), 'Meta columns missing'

def build_features(df, folder):
    ids = df['Id'].tolist()
    X_meta = df[meta_cols].astype(np.float32).values
    stats = []
    for i, id_ in enumerate(ids):
        if (i+1) % 1000 == 0:
            print(f'  Stats {i+1}/{len(ids)} ...', flush=True)
        stats.append(compute_img_stats(os.path.join(folder, f'{id_}.jpg')))
    X_stats = np.asarray(stats, dtype=np.float32)
    X = np.concatenate([X_meta, X_stats], axis=1)
    return ids, X

print('Building train features...')
tr_ids, X_tr = build_features(train_df, 'train')
print('Building test features...')
te_ids, X_te = build_features(test_df, 'test')

np.save('X_meta_stats_train.npy', X_tr)
np.save('X_meta_stats_test.npy', X_te)
pd.Series(tr_ids, name='Id').to_csv('ids_train_order.csv', index=False)
pd.Series(te_ids, name='Id').to_csv('ids_test_order.csv', index=False)
print('Saved X_meta_stats_train.npy', X_tr.shape, '| X_meta_stats_test.npy', X_te.shape)
print(f'Done in {time.time()-t0:.1f}s')

Building train features...


  Stats 1000/8920 ...


  Stats 2000/8920 ...


  Stats 3000/8920 ...


  Stats 4000/8920 ...


...

  Stats 6000/8920 ...


  Stats 7000/8920 ...


  Stats 8000/8920 ...


Building test features...


Saved X_meta_stats_train.npy (8920, 22) | X_meta_stats_test.npy (992, 22)
Done in 781.4s


In [10]:
# CPU fallback: 5-fold CV training on embeddings + meta-stats; blend and create submission
import os, glob, time, math, json
import numpy as np, pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def load_feature_set(name):
    if name == 'meta_stats':
        X_tr = np.load('X_meta_stats_train.npy') if os.path.exists('X_meta_stats_train.npy') else None
        X_te = np.load('X_meta_stats_test.npy') if os.path.exists('X_meta_stats_test.npy') else None
        return X_tr, X_te
    elif name.startswith('emb:'):
        tag = name.split(':',1)[1]
        tr_path = f"X_img_train_{tag}.npy"
        te_path = f"X_img_test_{tag}.npy"
        X_tr = np.load(tr_path) if os.path.exists(tr_path) else None
        X_te = np.load(te_path) if os.path.exists(te_path) else None
        return X_tr, X_te
    else:
        return None, None

def discover_embeddings():
    feats = []
    for tr_path in sorted(glob.glob('X_img_train_*.npy')):
        tag = tr_path[len('X_img_train_'):-len('.npy')]
        te_path = f"X_img_test_{tag}.npy"
        if os.path.exists(te_path):
            feats.append(f"emb:{tag}")
    return feats

# Try importing boosted trees; fallback to sklearn if unavailable
lgbm_ok = False; xgb_ok = False
try:
    import lightgbm as lgb
    lgbm_ok = True
except Exception as e:
    print('LightGBM not available:', e)
try:
    from xgboost import XGBRegressor
    xgb_ok = True
except Exception as e:
    print('XGBoost not available:', e)

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
folds_df = pd.read_csv('folds.csv')
id2fold = dict(zip(folds_df['Id'], folds_df['fold']))
train_df['fold'] = train_df['Id'].map(id2fold)
y = train_df['Pawpularity'].values.astype(float)

# Gather feature sets available
feature_sets = []
if os.path.exists('X_meta_stats_train.npy') and os.path.exists('X_meta_stats_test.npy'):
    feature_sets.append('meta_stats')
feature_sets += discover_embeddings()
print('Feature sets found:', feature_sets)
assert len(feature_sets) > 0, 'No feature sets found yet. Run extraction cells first.'

results = {}
oof_blend = np.zeros(len(train_df), dtype=float)
test_preds_for_blend = []
weights = []

for fs in feature_sets:
    X_tr, X_te = load_feature_set(fs)
    if X_tr is None or X_te is None:
        print(f'Skipping {fs}: files missing')
        continue
    print(f'Feature set {fs}: train {X_tr.shape}, test {X_te.shape}')

    oof = np.zeros(len(train_df), dtype=float)
    te_pred_accum = np.zeros(len(test_df), dtype=float)
    fold_rmses = []

    for fold in range(5):
        tr_idx = np.where(train_df['fold'].values != fold)[0]
        va_idx = np.where(train_df['fold'].values == fold)[0]
        X_tr_fold, y_tr_fold = X_tr[tr_idx], y[tr_idx]
        X_va_fold, y_va_fold = X_tr[va_idx], y[va_idx]

        model_name = None
        if lgbm_ok:
            model_name = 'lgbm'
            dtrain = lgb.Dataset(X_tr_fold, label=y_tr_fold)
            dvalid = lgb.Dataset(X_va_fold, label=y_va_fold)
            params = dict(objective='regression', metric='rmse', learning_rate=0.03,
                          num_leaves=64, min_data_in_leaf=20, feature_fraction=0.8,
                          bagging_fraction=0.8, bagging_freq=1, verbosity=-1)
            gbm = lgb.train(params, dtrain, num_boost_round=5000, valid_sets=[dvalid],
                            valid_names=['valid'], callbacks=[lgb.early_stopping(200), lgb.log_evaluation(100)])
            va_pred = gbm.predict(X_va_fold, num_iteration=gbm.best_iteration)
            te_pred = gbm.predict(X_te, num_iteration=gbm.best_iteration)
        elif xgb_ok:
            model_name = 'xgb'
            xgb = XGBRegressor(n_estimators=5000, learning_rate=0.03, max_depth=7, subsample=0.8, colsample_bytree=0.8,
                               reg_alpha=0.0, reg_lambda=1.0, tree_method='hist', random_state=42)
            xgb.fit(X_tr_fold, y_tr_fold,
                    eval_set=[(X_va_fold, y_va_fold)], eval_metric='rmse', verbose=100,
                    callbacks=[])
            va_pred = xgb.predict(X_va_fold)
            te_pred = xgb.predict(X_te)
        else:
            model_name = 'ridge'
            scaler = StandardScaler(with_mean=True, with_std=True)
            X_tr_s = scaler.fit_transform(X_tr_fold)
            X_va_s = scaler.transform(X_va_fold)
            X_te_s = scaler.transform(X_te)
            ridge = RidgeCV(alphas=[0.1, 0.3, 1.0, 3.0, 10.0], cv=5, scoring='neg_root_mean_squared_error')
            ridge.fit(X_tr_s, y_tr_fold)
            va_pred = ridge.predict(X_va_s)
            te_pred = ridge.predict(X_te_s)

        oof[va_idx] = va_pred
        fold_rmse = rmse(y_va_fold, va_pred)
        fold_rmses.append(fold_rmse)
        te_pred_accum += te_pred
        print(f'  {fs} fold {fold} [{model_name}] RMSE: {fold_rmse:.4f}')

    te_pred_mean = te_pred_accum / 5.0
    fs_oof_rmse = rmse(y, oof)
    results[fs] = {'oof_rmse': fs_oof_rmse, 'fold_rmses': fold_rmses}
    np.save(f'oof_{fs}.npy', oof)
    np.save(f'test_pred_{fs}.npy', te_pred_mean)
    print(f'{fs} OOF RMSE: {fs_oof_rmse:.5f}')

    test_preds_for_blend.append(te_pred_mean)
    # inverse-variance style weight; avoid div by zero
    w = 1.0 / max(fs_oof_rmse**2, 1e-6)
    weights.append(w)

# Blend available models
if len(test_preds_for_blend) == 0:
    raise RuntimeError('No successful models to blend.')
weights = np.array(weights, dtype=float)
weights = weights / weights.sum()
print('Blend weights:', {fs: float(w) for fs, w in zip(results.keys(), weights)})
stacked = np.vstack(test_preds_for_blend)
blend_test = (weights.reshape(-1,1) * stacked).sum(axis=0)

# Clip predictions and save submission
blend_test = np.clip(blend_test, 1.0, 100.0)
sub = pd.DataFrame({'Id': test_df['Id'], 'Pawpularity': blend_test})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv. Head:')
print(sub.head())
print('Results summary:', json.dumps(results, indent=2))

Feature sets found: ['meta_stats', 'emb:swin_tiny_patch4_window7_224_224', 'emb:tf_efficientnet_b0_224']
Feature set meta_stats: train (8920, 22), test (992, 22)
Training until validation scores don't improve for 200 rounds
[100]	valid's rmse: 20.4865


[200]	valid's rmse: 20.6486
Early stopping, best iteration is:
[62]	valid's rmse: 20.4324
  meta_stats fold 0 [lgbm] RMSE: 20.4324
Training until validation scores don't improve for 200 rounds


[100]	valid's rmse: 20.842
[200]	valid's rmse: 21.1366
Early stopping, best iteration is:
[5]	valid's rmse: 20.6156
  meta_stats fold 1 [lgbm] RMSE: 20.6156
Training until validation scores don't improve for 200 rounds


[100]	valid's rmse: 20.8778
[200]	valid's rmse: 21.0774


Early stopping, best iteration is:
[16]	valid's rmse: 20.695
  meta_stats fold 2 [lgbm] RMSE: 20.6950
Training until validation scores don't improve for 200 rounds
[100]	valid's rmse: 20.7922


...

Training until validation scores don't improve for 200 rounds


[100]	valid's rmse: 18.3521


[200]	valid's rmse: 18.31


[300]	valid's rmse: 18.3303


Early stopping, best iteration is:
[145]	valid's rmse: 18.298
  emb:tf_efficientnet_b0_224 fold 4 [lgbm] RMSE: 18.2980
emb:tf_efficientnet_b0_224 OOF RMSE: 18.08268
Blend weights: {'meta_stats': 0.27992766980776335, 'emb:swin_tiny_patch4_window7_224_224': 0.3566479893738934, 'emb:tf_efficientnet_b0_224': 0.3634243408183432}
Saved submission.csv. Head:
                                 Id  Pawpularity
0  ee51b99832f1ba868f646df93d2b6b81    49.616345
1  caddfb3f8bff9c4b95dbe022018eea21    33.025846
2  582eeabd4a448a53ebb79995888a4b0b    35.647525
3  afc1ad7f0c5eea880759d09e77f7deee    32.913339
4  d5bdf3446e86ce4ec67ce7a00f1cccc2    32.701493
Results summary: {
  "meta_stats": {
    "oof_rmse": 20.603775323132393,
    "fold_rmses": [
      20.432436381299492,
      20.61560497149953,
      20.694954573731803,
      20.615822958224896,
      20.65906246566223
    ]
  },
  "emb:swin_tiny_patch4_window7_224_224": {
    "oof_rmse": 18.25365466088792,
    "fold_rmses": [
      17.5942140123543

In [16]:
# Extract additional CPU-friendly embeddings (per expert ROI list) incl. OpenCLIP
import os, time, gc, math, subprocess, sys
import numpy as np, pandas as pd
import torch
import timm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from timm.data import resolve_data_config, create_transform

torch.set_num_threads(8)

def ensure_openclip():
    try:
        import open_clip  # noqa
    except Exception:
        print("Installing open-clip-torch (no-deps)...", flush=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-deps', '--upgrade-strategy', 'only-if-needed', 'open-clip-torch'], check=True)
    # ensure runtime deps without touching torch
    try:
        import ftfy  # noqa
    except Exception:
        print("Installing ftfy...", flush=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed', 'ftfy'], check=True)
    try:
        import regex  # noqa
    except Exception:
        print("Installing regex...", flush=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed', 'regex'], check=True)
    return True

class ImageDataset(Dataset):
    def __init__(self, ids, folder, transform):
        self.ids = ids
        self.folder = folder
        self.transform = transform
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        img_id = self.ids[idx]
        path = os.path.join(self.folder, f"{img_id}.jpg")
        img = Image.open(path).convert('RGB')
        img = self.transform(img)
        return img

def extract_timm_embeddings(model_name, img_size=224, batch_size=128, num_workers=8):
    tr_out = f"X_img_train_{model_name.replace('/', '_')}_{img_size}.npy"
    te_out = f"X_img_test_{model_name.replace('/', '_')}_{img_size}.npy"
    if os.path.exists(tr_out) and os.path.exists(te_out):
        print(f"[Skip] {model_name} exists: {tr_out}, {te_out}")
        return
    t0 = time.time()
    print(f"\n[Emb] {model_name} @ {img_size}", flush=True)
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    train_ids = train_df['Id'].tolist()
    test_ids = test_df['Id'].tolist()

    model = timm.create_model(model_name, pretrained=True, num_classes=0, global_pool='avg')
    model.eval().to('cpu')

    cfg = resolve_data_config({}, model=model)
    cfg['input_size'] = (3, img_size, img_size)
    transform = create_transform(**cfg, is_training=False)

    train_ds = ImageDataset(train_ids, 'train', transform)
    test_ds = ImageDataset(test_ids, 'test', transform)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False)

    with torch.no_grad():
        sample = next(iter(train_loader))[:1]
        emb_dim = model(sample).shape[1]
    print(f"Embedding dim: {emb_dim}")

    def run_loader(loader, n_items):
        X = np.zeros((n_items, emb_dim), dtype=np.float32)
        i0 = 0
        with torch.no_grad():
            for i, xb in enumerate(loader):
                feats = model(xb).cpu().numpy().astype(np.float32)
                X[i0:i0+feats.shape[0]] = feats
                i0 += feats.shape[0]
                if (i+1) % 20 == 0:
                    print(f"  Batches {i+1}/{math.ceil(n_items/loader.batch_size)} | rows {i0}/{n_items} | elapsed {time.time()-t0:.1f}s", flush=True)
        return X

    X_tr = run_loader(train_loader, len(train_ds))
    X_te = run_loader(test_loader, len(test_ds))
    np.save(tr_out, X_tr); np.save(te_out, X_te)
    print(f"Saved {tr_out} {X_tr.shape}, {te_out} {X_te.shape} | time {time.time()-t0:.1f}s")
    del model, X_tr, X_te, train_loader, test_loader, train_ds, test_ds
    gc.collect()

def extract_openclip_vitb32(img_size=224, batch_size=128, num_workers=8):
    tag = f"openclip_vit_b32_{img_size}"
    tr_out = f"X_img_train_{tag}.npy"
    te_out = f"X_img_test_{tag}.npy"
    if os.path.exists(tr_out) and os.path.exists(te_out):
        print(f"[Skip] OpenCLIP ViT-B/32 exists: {tr_out}, {te_out}")
        return
    ensure_openclip()
    import open_clip
    from torchvision import transforms as T

    t0 = time.time()
    print(f"\n[Emb] OpenCLIP ViT-B/32 @ {img_size}", flush=True)
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', device='cpu')
    model.eval()
    preprocess = T.Compose([
        T.Resize(img_size, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(img_size),
        T.ToTensor(),
        T.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711)),
    ])

    class OpenClipDS(Dataset):
        def __init__(self, ids, folder):
            self.ids = ids; self.folder = folder
        def __len__(self): return len(self.ids)
        def __getitem__(self, idx):
            img = Image.open(os.path.join(self.folder, f"{self.ids[idx]}.jpg")).convert('RGB')
            return preprocess(img)

    train_df = pd.read_csv('train.csv'); test_df = pd.read_csv('test.csv')
    train_ids = train_df['Id'].tolist(); test_ids = test_df['Id'].tolist()
    train_loader = DataLoader(OpenClipDS(train_ids, 'train'), batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(OpenClipDS(test_ids, 'test'), batch_size=batch_size, shuffle=False, num_workers=num_workers)

    with torch.no_grad():
        sample = next(iter(train_loader))[:1]
        feats = model.encode_image(sample)
        emb_dim = feats.shape[1]
    print(f"Embedding dim: {emb_dim}")

    def run_loader(loader, n_items):
        X = np.zeros((n_items, emb_dim), dtype=np.float32)
        i0 = 0
        with torch.no_grad():
            for i, xb in enumerate(loader):
                feats = model.encode_image(xb).float().cpu().numpy().astype(np.float32)
                X[i0:i0+feats.shape[0]] = feats
                i0 += feats.shape[0]
                if (i+1) % 20 == 0:
                    print(f"  Batches {i+1}/{math.ceil(n_items/loader.batch_size)} | rows {i0}/{n_items} | elapsed {time.time()-t0:.1f}s", flush=True)
        return X

    X_tr = run_loader(train_loader, len(train_ids))
    X_te = run_loader(test_loader, len(test_ids))
    np.save(tr_out, X_tr); np.save(te_out, X_te)
    print(f"Saved {tr_out} {X_tr.shape}, {te_out} {X_te.shape} | time {time.time()-t0:.1f}s")
    del model, X_tr, X_te, train_loader, test_loader
    gc.collect()

# Queue models (fast ROI on CPU)
models = [
    ('convnext_tiny_in22k', 224),
    ('tf_efficientnetv2_s_in21k', 224),
    ('swin_small_patch4_window7_224', 224),
    ('vit_base_patch16_224', 224),
    ('deit3_small_patch16_224', 224),
]

for name, sz in models:
    try:
        extract_timm_embeddings(name, img_size=sz, batch_size=128, num_workers=8)
    except Exception as e:
        print(f"[Warn] Failed {name}: {e}")

# OpenCLIP ViT-B/32
try:
    extract_openclip_vitb32(img_size=224, batch_size=128, num_workers=8)
except Exception as e:
    print(f"[Warn] Failed OpenCLIP ViT-B/32: {e}")
print('Additional embeddings extraction done.')

[Skip] convnext_tiny_in22k exists: X_img_train_convnext_tiny_in22k_224.npy, X_img_test_convnext_tiny_in22k_224.npy
[Skip] tf_efficientnetv2_s_in21k exists: X_img_train_tf_efficientnetv2_s_in21k_224.npy, X_img_test_tf_efficientnetv2_s_in21k_224.npy
[Skip] swin_small_patch4_window7_224 exists: X_img_train_swin_small_patch4_window7_224_224.npy, X_img_test_swin_small_patch4_window7_224_224.npy
[Skip] vit_base_patch16_224 exists: X_img_train_vit_base_patch16_224_224.npy, X_img_test_vit_base_patch16_224_224.npy
[Skip] deit3_small_patch16_224 exists: X_img_train_deit3_small_patch16_224_224.npy, X_img_test_deit3_small_patch16_224_224.npy
Installing open-clip-torch (no-deps)...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.0 MB/s eta 0:00:00
Installing ftfy...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 KB 3.4 MB/s eta 0:00:00


Installing regex...


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-clip-torch 3.2.0 requires regex, which is not installed.


...

Embedding dim: 512


  Batches 20/70 | rows 2560/8920 | elapsed 37.9s


  Batches 40/70 | rows 5120/8920 | elapsed 67.5s


  Batches 60/70 | rows 7680/8920 | elapsed 97.1s


Saved X_img_train_openclip_vit_b32_224.npy (8920, 512), X_img_test_openclip_vit_b32_224.npy (992, 512) | time 124.5s
Additional embeddings extraction done.


In [20]:
# Improved training: L2+PCA for embeddings, LightGBM per set, try raw vs sqrt target per-set, NNLS blend
import os, glob, time, math, json
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def load_feature_set(name):
    if name == 'meta_stats':
        X_tr = np.load('X_meta_stats_train.npy') if os.path.exists('X_meta_stats_train.npy') else None
        X_te = np.load('X_meta_stats_test.npy') if os.path.exists('X_meta_stats_test.npy') else None
        return X_tr, X_te
    elif name.startswith('emb:'):
        tag = name.split(':',1)[1]
        tr_path = f"X_img_train_{tag}.npy"
        te_path = f"X_img_test_{tag}.npy"
        X_tr = np.load(tr_path) if os.path.exists(tr_path) else None
        X_te = np.load(te_path) if os.path.exists(te_path) else None
        return X_tr, X_te
    else:
        return None, None

def discover_feature_sets():
    feats = []
    if os.path.exists('X_meta_stats_train.npy') and os.path.exists('X_meta_stats_test.npy'):
        feats.append('meta_stats')
    for tr_path in sorted(glob.glob('X_img_train_*.npy')):
        tag = tr_path[len('X_img_train_'):-len('.npy')]
        te_path = f"X_img_test_{tag}.npy"
        if os.path.exists(te_path):
            feats.append(f"emb:{tag}")
    return feats

# Import LightGBM if available
lgbm_ok = False
try:
    import lightgbm as lgb
    lgbm_ok = True
except Exception as e:
    print('LightGBM not available:', e)

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
folds_df = pd.read_csv('folds.csv')
id2fold = dict(zip(folds_df['Id'], folds_df['fold']))
train_df['fold'] = train_df['Id'].map(id2fold)
y = train_df['Pawpularity'].values.astype(float)
y_sqrt = np.sqrt(np.clip(y, 0, None))

feature_sets = discover_feature_sets()
print('Feature sets found:', feature_sets)
assert len(feature_sets) > 0, 'No feature sets found.'

def train_feature_set(fs, X_tr, X_te, target_mode='raw'):
    use_pca = fs.startswith('emb:') and X_tr.shape[1] >= 512
    n_comp = 512 if X_tr.shape[1] >= 512 else X_tr.shape[1]
    oof = np.zeros(len(train_df), dtype=float)
    te_pred_accum = np.zeros(len(test_df), dtype=float)
    fold_rmses = []

    for fold in range(5):
        tr_idx = np.where(train_df['fold'].values != fold)[0]
        va_idx = np.where(train_df['fold'].values == fold)[0]
        X_tr_fold = X_tr[tr_idx]
        X_va_fold = X_tr[va_idx]
        X_te_fold = X_te
        y_tr_fold_raw = y[tr_idx]
        y_va_fold_raw = y[va_idx]
        if target_mode == 'sqrt':
            y_tr_fold = y_sqrt[tr_idx]
            y_va_fold_eval = y_va_fold_raw  # evaluate in raw space after inverse-transform
        else:
            y_tr_fold = y_tr_fold_raw
            y_va_fold_eval = y_va_fold_raw

        if use_pca:
            def l2norm(a):
                n = np.linalg.norm(a, axis=1, keepdims=True) + 1e-12
                return a / n
            X_tr_fold = l2norm(X_tr_fold)
            X_va_fold = l2norm(X_va_fold)
            X_te_fold = l2norm(X_te_fold)
            pca = PCA(n_components=min(n_comp, X_tr.shape[1]), whiten=False, random_state=42)
            X_tr_fold = pca.fit_transform(X_tr_fold)
            X_va_fold = pca.transform(X_va_fold)
            X_te_fold = pca.transform(X_te_fold)
            print(f'  Fold {fold}: PCA -> {X_tr_fold.shape[1]} dims')

        if lgbm_ok:
            dtrain = lgb.Dataset(X_tr_fold, label=y_tr_fold)
            if target_mode == 'sqrt':
                dvalid = lgb.Dataset(X_va_fold, label=np.sqrt(np.clip(y_va_fold_raw, 0, None)))
            else:
                dvalid = lgb.Dataset(X_va_fold, label=y_va_fold_raw)
            params = dict(objective='regression', metric='rmse', learning_rate=0.03,
                          num_leaves=64, min_data_in_leaf=40, feature_fraction=0.8,
                          bagging_fraction=0.8, bagging_freq=1, lambda_l2=1.0, verbosity=-1)
            gbm = lgb.train(params, dtrain, num_boost_round=10000, valid_sets=[dvalid],
                            valid_names=['valid'], callbacks=[lgb.early_stopping(300), lgb.log_evaluation(100)])
            va_pred = gbm.predict(X_va_fold, num_iteration=gbm.best_iteration)
            te_pred = gbm.predict(X_te_fold, num_iteration=gbm.best_iteration)
        else:
            scaler = StandardScaler(with_mean=True, with_std=True)
            X_tr_s = scaler.fit_transform(X_tr_fold)
            X_va_s = scaler.transform(X_va_fold)
            X_te_s = scaler.transform(X_te_fold)
            from sklearn.linear_model import RidgeCV
            ridge = RidgeCV(alphas=[0.1, 0.3, 1.0, 3.0, 10.0], cv=5, scoring='neg_root_mean_squared_error')
            ridge.fit(X_tr_s, y_tr_fold)
            va_pred = ridge.predict(X_va_s)
            te_pred = ridge.predict(X_te_s)

        if target_mode == 'sqrt':
            va_pred = np.clip(va_pred, 0, None) ** 2
            te_pred = np.clip(te_pred, 0, None) ** 2
        va_pred = np.clip(va_pred, 1.0, 100.0)
        te_pred = np.clip(te_pred, 1.0, 100.0)

        oof[va_idx] = va_pred
        fold_rmses.append(rmse(y_va_fold_eval, va_pred))
        te_pred_accum += te_pred
        print(f'  {fs} [{target_mode}] fold {fold} RMSE: {fold_rmses[-1]:.4f}')

    te_pred_mean = te_pred_accum / 5.0
    fs_oof_rmse = rmse(y, oof)
    return {'oof': oof, 'test_pred': te_pred_mean, 'oof_rmse': fs_oof_rmse, 'fold_rmses': fold_rmses, 'used_pca': use_pca, 'n_comp': int(n_comp) if use_pca else int(X_tr.shape[1]), 'target': target_mode}

results = {}
oof_matrix = []
test_matrix = []
names_used = []

for fs in feature_sets:
    X_tr, X_te = load_feature_set(fs)
    if X_tr is None or X_te is None:
        print(f'Skipping {fs}: files missing')
        continue
    print(f'Feature set {fs}: train {X_tr.shape}, test {X_te.shape}')

    res_raw = train_feature_set(fs, X_tr, X_te, target_mode='raw')
    res_sqrt = train_feature_set(fs, X_tr, X_te, target_mode='sqrt') if fs.startswith('emb:') else None
    cand = [res_raw] + ([res_sqrt] if res_sqrt is not None else [])
    best = min(cand, key=lambda d: d['oof_rmse'])
    print(f"{fs} best target: {best['target']} | OOF RMSE: {best['oof_rmse']:.5f}")

    # Save best
    np.save(f'oof_{fs}.npy', best['oof'])
    np.save(f'test_pred_{fs}.npy', best['test_pred'])
    results[fs] = {k: best[k] for k in ['oof_rmse','fold_rmses','used_pca','n_comp','target']}

    oof_matrix.append(best['oof'])
    test_matrix.append(best['test_pred'])
    names_used.append(fs)

# Blend via non-negative least squares (LinearRegression positive=True, no intercept)
if len(oof_matrix) == 0:
    raise RuntimeError('No successful models to blend.')
P = np.vstack(oof_matrix).T
reg = LinearRegression(fit_intercept=False, positive=True)
reg.fit(P, y)
w = reg.coef_.copy()
if w.sum() == 0:
    w = np.ones_like(w)
w = w / w.sum()
print('NNLS-like blend weights:', {n: float(wi) for n, wi in zip(names_used, w)})
Tstack = np.vstack(test_matrix)
blend_test = (w.reshape(-1,1) * Tstack).sum(axis=0)
blend_test = np.clip(blend_test, 1.0, 100.0)
sub = pd.DataFrame({'Id': test_df['Id'], 'Pawpularity': blend_test})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv. Head:')
print(sub.head())
print('Results summary:', json.dumps(results, indent=2))

Feature sets found: ['meta_stats', 'emb:convnext_tiny_in22k_224', 'emb:deit3_small_patch16_224_224', 'emb:openclip_vit_b32_224', 'emb:swin_small_patch4_window7_224_224', 'emb:swin_tiny_patch4_window7_224_224', 'emb:tf_efficientnet_b0_224', 'emb:tf_efficientnetv2_s_in21k_224', 'emb:vit_base_patch16_224_224']
Feature set meta_stats: train (8920, 22), test (992, 22)
Training until validation scores don't improve for 300 rounds
[100]	valid's rmse: 20.5344


[200]	valid's rmse: 20.6577
[300]	valid's rmse: 20.8641


Early stopping, best iteration is:
[40]	valid's rmse: 20.4286
  meta_stats [raw] fold 0 RMSE: 20.4286
Training until validation scores don't improve for 300 rounds
[100]	valid's rmse: 20.9231


[200]	valid's rmse: 21.2493
[300]	valid's rmse: 21.4906
Early stopping, best iteration is:
[2]	valid's rmse: 20.6147
  meta_stats [raw] fold 1 RMSE: 20.6147
Training until validation scores don't improve for 300 rounds


[100]	valid's rmse: 20.8299
[200]	valid's rmse: 21.0454


...

[100]	valid's rmse: 1.50918


[200]	valid's rmse: 1.50462


[300]	valid's rmse: 1.50589


[400]	valid's rmse: 1.50716


[500]	valid's rmse: 1.50666
Early stopping, best iteration is:
[212]	valid's rmse: 1.50284
  emb:vit_base_patch16_224_224 [sqrt] fold 4 RMSE: 18.6707
emb:vit_base_patch16_224_224 best target: raw | OOF RMSE: 18.24591
NNLS-like blend weights: {'meta_stats': 0.0, 'emb:convnext_tiny_in22k_224': 0.032384446586774765, 'emb:deit3_small_patch16_224_224': 0.05806099820648654, 'emb:openclip_vit_b32_224': 0.24248823516498855, 'emb:swin_small_patch4_window7_224_224': 0.16702355124203574, 'emb:swin_tiny_patch4_window7_224_224': 0.028105946831819956, 'emb:tf_efficientnet_b0_224': 0.19094628513304182, 'emb:tf_efficientnetv2_s_in21k_224': 0.10936125649893741, 'emb:vit_base_patch16_224_224': 0.17162928033591524}
Saved submission.csv. Head:
                                 Id  Pawpularity
0  ee51b99832f1ba868f646df93d2b6b81    51.831425
1  caddfb3f8bff9c4b95dbe022018eea21    35.857884
2  582eeabd4a448a53ebb79995888a4b0b    35.430708
3  afc1ad7f0c5eea880759d09e77f7deee    33.104743
4  d5bdf3446e86ce4ec6

In [18]:
# Compute blended OOF RMSE for current saved OOFs
import glob, json
import numpy as np, pandas as pd
from sklearn.linear_model import LinearRegression

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

train_df = pd.read_csv('train.csv')
y = train_df['Pawpularity'].values.astype(float)

# discover OOF files
oof_files = sorted(glob.glob('oof_*.npy'))
names = [f[4:-4] for f in oof_files]  # strip 'oof_' and '.npy'
if len(oof_files) == 0:
    raise SystemExit('No oof_*.npy found.')
print('OOF files:', names)

O = np.vstack([np.load(f) for f in oof_files]).T  # (n_samples, n_models)
single_scores = {n: rmse(y, O[:,i]) for i, n in enumerate(names)}
print('Single OOF RMSEs:', json.dumps(single_scores, indent=2))

# NNLS-like blend (non-negative, no intercept)
reg = LinearRegression(fit_intercept=False, positive=True)
reg.fit(O, y)
w = reg.coef_.copy()
w = w / (w.sum() if w.sum() > 0 else 1.0)
blend_oof = O @ w
blend_rmse = rmse(y, blend_oof)
print('NNLS weights:', {n: float(wi) for n, wi in zip(names, w)})
print(f'NNLS blend OOF RMSE: {blend_rmse:.5f}')

# Inverse-variance weighting
invw = np.array([1.0 / max(single_scores[n]**2, 1e-6) for n in names], dtype=float)
invw = invw / invw.sum()
blend_iv = O @ invw
blend_iv_rmse = rmse(y, blend_iv)
print('Inv-var weights:', {n: float(wi) for n, wi in zip(names, invw)})
print(f'Inv-var blend OOF RMSE: {blend_iv_rmse:.5f}')

OOF files: ['emb:convnext_tiny_in22k_224', 'emb:deit3_small_patch16_224_224', 'emb:openclip_vit_b32_224', 'emb:swin_small_patch4_window7_224_224', 'emb:swin_tiny_patch4_window7_224_224', 'emb:tf_efficientnet_b0_224', 'emb:tf_efficientnetv2_s_in21k_224', 'emb:vit_base_patch16_224_224', 'meta_stats']
Single OOF RMSEs: {
  "emb:convnext_tiny_in22k_224": 18.343681791120346,
  "emb:deit3_small_patch16_224_224": 18.519840874368242,
  "emb:openclip_vit_b32_224": 18.275847412336976,
  "emb:swin_small_patch4_window7_224_224": 18.173627077168888,
  "emb:swin_tiny_patch4_window7_224_224": 18.33451618004236,
  "emb:tf_efficientnet_b0_224": 18.212714967463256,
  "emb:tf_efficientnetv2_s_in21k_224": 18.284960028331636,
  "emb:vit_base_patch16_224_224": 18.316501053056516,
  "meta_stats": 20.603775323132393
}
NNLS weights: {'emb:convnext_tiny_in22k_224': 0.011721418495030226, 'emb:deit3_small_patch16_224_224': 0.05101623467959047, 'emb:openclip_vit_b32_224': 0.2438208130537412, 'emb:swin_small_patch4

In [29]:
# Add HGBR base learner per feature set with per-fold L2+PCA; save OOF/test for stacking
import os, glob, time, math, json
import numpy as np, pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def load_feature_set(name):
    if name == 'meta_stats':
        X_tr = np.load('X_meta_stats_train.npy') if os.path.exists('X_meta_stats_train.npy') else None
        X_te = np.load('X_meta_stats_test.npy') if os.path.exists('X_meta_stats_test.npy') else None
        return X_tr, X_te
    elif name.startswith('emb:'):
        tag = name.split(':',1)[1]
        tr_path = f"X_img_train_{tag}.npy"
        te_path = f"X_img_test_{tag}.npy"
        X_tr = np.load(tr_path) if os.path.exists(tr_path) else None
        X_te = np.load(te_path) if os.path.exists(te_path) else None
        return X_tr, X_te
    else:
        return None, None

def discover_feature_sets():
    feats = []
    if os.path.exists('X_meta_stats_train.npy') and os.path.exists('X_meta_stats_test.npy'):
        feats.append('meta_stats')
    for tr_path in sorted(glob.glob('X_img_train_*.npy')):
        tag = tr_path[len('X_img_train_'):-len('.npy')]
        te_path = f"X_img_test_{tag}.npy"
        if os.path.exists(te_path):
            feats.append(f"emb:{tag}")
    return feats

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
folds_df = pd.read_csv('folds.csv')
id2fold = dict(zip(folds_df['Id'], folds_df['fold']))
train_df['fold'] = train_df['Id'].map(id2fold)
y = train_df['Pawpularity'].values.astype(float)
y_sqrt = np.sqrt(np.clip(y, 0, None))

feature_sets = discover_feature_sets()
print('Feature sets:', feature_sets)
assert len(feature_sets) > 0, 'No feature sets found.'

def l2norm(a):
    n = np.linalg.norm(a, axis=1, keepdims=True) + 1e-12
    return a / n

def train_hgbr_for_fs(fs, X_tr, X_te, target_mode='raw'):
    use_pca = fs.startswith('emb:') and X_tr.shape[1] >= 512
    n_comp = 512 if X_tr.shape[1] >= 512 else X_tr.shape[1]
    oof = np.zeros(len(train_df), dtype=float)
    te_pred_accum = np.zeros(len(test_df), dtype=float)
    fold_rmses = []
    t0 = time.time()

    for fold in range(5):
        tr_idx = np.where(train_df['fold'].values != fold)[0]
        va_idx = np.where(train_df['fold'].values == fold)[0]
        X_tr_fold = X_tr[tr_idx].copy()
        X_va_fold = X_tr[va_idx].copy()
        X_te_fold = X_te.copy()

        y_tr_fold_raw = y[tr_idx]
        y_va_fold_raw = y[va_idx]

        if target_mode == 'sqrt':
            y_tr_fold = y_sqrt[tr_idx]
            eval_y = y_va_fold_raw
        else:
            y_tr_fold = y_tr_fold_raw
            eval_y = y_va_fold_raw

        if use_pca:
            X_tr_fold = l2norm(X_tr_fold)
            X_va_fold = l2norm(X_va_fold)
            X_te_fold = l2norm(X_te_fold)
            pca = PCA(n_components=min(n_comp, X_tr.shape[1]), whiten=False, random_state=42)
            X_tr_fold = pca.fit_transform(X_tr_fold)
            X_va_fold = pca.transform(X_va_fold)
            X_te_fold = pca.transform(X_te_fold)
            print(f'  {fs} [{target_mode}] fold {fold}: PCA -> {X_tr_fold.shape[1]} dims', flush=True)

        model = HistGradientBoostingRegressor(
            learning_rate=0.04,
            max_iter=2000,
            max_leaf_nodes=31,
            min_samples_leaf=30,
            l2_regularization=1.0,
            validation_fraction=0.12,
            early_stopping=True,
            random_state=42
        )
        model.fit(X_tr_fold, y_tr_fold)
        va_pred = model.predict(X_va_fold)
        te_pred = model.predict(X_te_fold)

        if target_mode == 'sqrt':
            va_pred = np.clip(va_pred, 0, None) ** 2
            te_pred = np.clip(te_pred, 0, None) ** 2

        va_pred = np.clip(va_pred, 1.0, 100.0)
        te_pred = np.clip(te_pred, 1.0, 100.0)

        oof[va_idx] = va_pred
        f_rmse = rmse(eval_y, va_pred)
        fold_rmses.append(f_rmse)
        te_pred_accum += te_pred
        print(f'    {fs} [HGBR|{target_mode}] fold {fold} RMSE: {f_rmse:.4f} | elapsed {time.time()-t0:.1f}s', flush=True)

    te_mean = te_pred_accum / 5.0
    fs_oof_rmse = rmse(y, oof)
    return {'oof': oof, 'test_pred': te_mean, 'oof_rmse': fs_oof_rmse, 'fold_rmses': fold_rmses, 'used_pca': use_pca, 'n_comp': int(n_comp) if use_pca else int(X_tr.shape[1]), 'target': target_mode}

results_hgbr = {}
for fs in feature_sets:
    X_tr, X_te = load_feature_set(fs)
    if X_tr is None or X_te is None:
        print(f'[Skip] {fs}: features missing')
        continue
    print(f'=== HGBR training for {fs}: train {X_tr.shape}, test {X_te.shape} ===', flush=True)
    res_raw = train_hgbr_for_fs(fs, X_tr, X_te, target_mode='raw')
    res_sqrt = train_hgbr_for_fs(fs, X_tr, X_te, target_mode='sqrt') if fs.startswith('emb:') else None
    best = res_raw if (res_sqrt is None or res_raw['oof_rmse'] <= res_sqrt['oof_rmse']) else res_sqrt
    results_hgbr[fs] = {k: best[k] for k in ['oof_rmse','fold_rmses','used_pca','n_comp','target']}
    np.save(f'oof_hgb_{fs}.npy', best['oof'])
    np.save(f'test_pred_hgb_{fs}.npy', best['test_pred'])
    print(f'>>> {fs} [HGBR] best target: {best["target"]} | OOF RMSE: {best["oof_rmse"]:.5f}', flush=True)

print('HGBR results:', json.dumps(results_hgbr, indent=2))

Feature sets: ['meta_stats', 'emb:convnext_tiny_in22k_224', 'emb:deit3_small_patch16_224_224', 'emb:openclip_vit_b16_224', 'emb:openclip_vit_b32_224', 'emb:swin_small_patch4_window7_224_224', 'emb:swin_tiny_patch4_window7_224_224', 'emb:tf_efficientnet_b0_224', 'emb:tf_efficientnetv2_s_in21k_224', 'emb:vit_base_patch16_224_224']
=== HGBR training for meta_stats: train (8920, 39), test (992, 39) ===


    meta_stats [HGBR|raw] fold 0 RMSE: 20.4575 | elapsed 0.1s


    meta_stats [HGBR|raw] fold 1 RMSE: 20.6736 | elapsed 0.2s


    meta_stats [HGBR|raw] fold 2 RMSE: 20.7505 | elapsed 0.3s


    meta_stats [HGBR|raw] fold 3 RMSE: 20.6603 | elapsed 0.4s


...

    emb:vit_base_patch16_224_224 [HGBR|sqrt] fold 3 RMSE: 18.6520 | elapsed 19.7s


  emb:vit_base_patch16_224_224 [sqrt] fold 4: PCA -> 512 dims


    emb:vit_base_patch16_224_224 [HGBR|sqrt] fold 4 RMSE: 18.9193 | elapsed 25.1s


>>> emb:vit_base_patch16_224_224 [HGBR] best target: raw | OOF RMSE: 18.45844


HGBR results: {
  "meta_stats": {
    "oof_rmse": 20.646863559898133,
    "fold_rmses": [
      20.45747953581059,
      20.67361744259402,
      20.750477198553718,
      20.6603480073514,
      20.691197728305713
    ],
    "used_pca": false,
    "n_comp": 39,
    "target": "raw"
  },
  "emb:convnext_tiny_in22k_224": {
    "oof_rmse": 18.45852255637422,
    "fold_rmses": [
      18.089962761292096,
      18.678476519274646,
      18.610188004644606,
      18.281480274905174,
      18.625289754118874
    ],
    "used_pca": true,
    "n_comp": 512,
    "target": "raw"
  },
  "emb:deit3_small_patch16_224_224": {
    "oof_rmse": 18.696130405766517,
    "fold_rmses": [
      18.29185240600516,
      18.823628435869978,
      19.047284746421806,
      18.542782911689756,
      18.76623975363846
    ],
    "used_pca": false,
    "n_comp": 384,
    "target": "raw"
  },
  "emb:openclip_vit_b16_224": {
    "oof_rmse": 18.325641171999695,
    "fold_rmses": [
      17.783822615909695,
      18.3

In [30]:
# Level-2 RidgeCV stacker with nested CV over LGBM + HGBR OOFs
import glob, json, time, os
import numpy as np, pandas as pd
from sklearn.linear_model import RidgeCV

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

# Load target and folds
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
folds_df = pd.read_csv('folds.csv')
id2fold = dict(zip(folds_df['Id'], folds_df['fold']))
train_df['fold'] = train_df['Id'].map(id2fold)
y = train_df['Pawpularity'].values.astype(float)

# Discover base OOF/test files (exclude previous L2 artifacts to avoid leakage)
all_oof = sorted(glob.glob('oof_*.npy'))
oof_lgb_files = [f for f in all_oof if not os.path.basename(f).startswith('oof_L2_')]
oof_hgb_files = sorted(glob.glob('oof_hgb_*.npy'))
def name_from(path, prefix):
    return path[len(prefix):-4]
names_lgb = [name_from(p, 'oof_') for p in oof_lgb_files]
names_hgb = [name_from(p, 'oof_hgb_') for p in oof_hgb_files]
print('Base models (LGB):', names_lgb)
print('Base models (HGB):', names_hgb)
assert len(oof_lgb_files) > 0 or len(oof_hgb_files) > 0, 'No base OOF files found.'

# Load matrices
cols = []
O_list = []
T_list = []
for p, n in zip(oof_lgb_files, names_lgb):
    O_list.append(np.load(p))
    T_list.append(np.load(f'test_pred_{n}.npy'))
    cols.append(f'lgb:{n}')
for p, n in zip(oof_hgb_files, names_hgb):
    O_list.append(np.load(p))
    T_list.append(np.load(f'test_pred_hgb_{n}.npy'))
    cols.append(f'hgb:{n}')
O = np.vstack(O_list).T  # (n_samples, n_models)
T = np.vstack(T_list).T  # (n_test, n_models)
print('L2 feature matrix shapes:', O.shape, T.shape)

# Row-wise aggregation features (simple and cheap)
def add_row_aggs(M, prefix):
    mean = M.mean(axis=1, keepdims=True)
    std = M.std(axis=1, keepdims=True)
    vmin = M.min(axis=1, keepdims=True)
    vmax = M.max(axis=1, keepdims=True)
    return np.hstack([M, mean, std, vmin, vmax]), [f'{prefix}_mean', f'{prefix}_std', f'{prefix}_min', f'{prefix}_max']

# Use combined columns for aggs
O_ext, agg_names = add_row_aggs(O, 'agg')
T_ext, _ = add_row_aggs(T, 'agg')
cols_ext = cols + agg_names

# Nested CV stacker
alphas = np.logspace(-4, 2, 20)
meta_oof = np.zeros(len(train_df), dtype=float)
meta_test_accum = np.zeros(len(test_df), dtype=float)
coefs = []
t0 = time.time()
for fold in range(5):
    tr_idx = np.where(train_df['fold'].values != fold)[0]
    va_idx = np.where(train_df['fold'].values == fold)[0]
    X_tr, X_va = O_ext[tr_idx], O_ext[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]
    ridge = RidgeCV(alphas=alphas, fit_intercept=True, cv=5, scoring='neg_root_mean_squared_error')
    ridge.fit(X_tr, y_tr)
    va_pred = ridge.predict(X_va)
    meta_oof[va_idx] = va_pred
    fold_rmse = rmse(y_va, va_pred)
    meta_test_accum += ridge.predict(T_ext)
    coefs.append(ridge.coef_.copy())
    print(f'L2 Ridge fold {fold} RMSE: {fold_rmse:.5f} | alpha: {ridge.alpha_:.5g} | elapsed {time.time()-t0:.1f}s', flush=True)

meta_rmse = rmse(y, meta_oof)
meta_test = meta_test_accum / 5.0
print(f'L2 Ridge meta OOF RMSE: {meta_rmse:.5f}')

# Save artifacts and submission
np.save('oof_L2_ridge.npy', meta_oof)
np.save('test_pred_L2_ridge.npy', meta_test)
sub = pd.DataFrame({'Id': test_df['Id'], 'Pawpularity': np.clip(meta_test, 1.0, 100.0)})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv. Head:')
print(sub.head())

# Report average coefficients per feature for inspection
avg_coef = np.mean(np.vstack(coefs), axis=0)
coef_report = {name: float(c) for name, c in zip(cols_ext, avg_coef)}
print('Average L2 coefficients (top 12 by abs):')
top = sorted(coef_report.items(), key=lambda kv: abs(kv[1]), reverse=True)[:12]
print(json.dumps(dict(top), indent=2))

# Also print single-model RMSEs for reference
single_scores = {cols[i]: rmse(y, O[:, i]) for i in range(len(cols))}
print('Single base OOF RMSEs (subset):', json.dumps(dict(list(single_scores.items())[:8]), indent=2))

Base models (LGB): ['emb:convnext_tiny_in22k_224', 'emb:deit3_small_patch16_224_224', 'emb:openclip_vit_b16_224', 'emb:openclip_vit_b32_224', 'emb:swin_small_patch4_window7_224_224', 'emb:swin_tiny_patch4_window7_224_224', 'emb:tf_efficientnet_b0_224', 'emb:tf_efficientnetv2_s_in21k_224', 'emb:vit_base_patch16_224_224', 'hgb_emb:convnext_tiny_in22k_224', 'hgb_emb:deit3_small_patch16_224_224', 'hgb_emb:openclip_vit_b16_224', 'hgb_emb:openclip_vit_b32_224', 'hgb_emb:swin_small_patch4_window7_224_224', 'hgb_emb:swin_tiny_patch4_window7_224_224', 'hgb_emb:tf_efficientnet_b0_224', 'hgb_emb:tf_efficientnetv2_s_in21k_224', 'hgb_emb:vit_base_patch16_224_224', 'hgb_meta_stats', 'meta_stats']
Base models (HGB): ['emb:convnext_tiny_in22k_224', 'emb:deit3_small_patch16_224_224', 'emb:openclip_vit_b16_224', 'emb:openclip_vit_b32_224', 'emb:swin_small_patch4_window7_224_224', 'emb:swin_tiny_patch4_window7_224_224', 'emb:tf_efficientnet_b0_224', 'emb:tf_efficientnetv2_s_in21k_224', 'emb:vit_base_patc

L2 Ridge fold 0 RMSE: 16.93826 | alpha: 100 | elapsed 0.2s


L2 Ridge fold 1 RMSE: 17.69146 | alpha: 100 | elapsed 0.5s


L2 Ridge fold 2 RMSE: 17.89423 | alpha: 100 | elapsed 0.8s


L2 Ridge fold 3 RMSE: 17.24341 | alpha: 100 | elapsed 1.0s


L2 Ridge fold 4 RMSE: 17.69985 | alpha: 100 | elapsed 1.2s


L2 Ridge meta OOF RMSE: 17.49695
Saved submission.csv. Head:
                                 Id  Pawpularity
0  ee51b99832f1ba868f646df93d2b6b81    54.325589
1  caddfb3f8bff9c4b95dbe022018eea21    39.015959
2  582eeabd4a448a53ebb79995888a4b0b    32.744878
3  afc1ad7f0c5eea880759d09e77f7deee    31.604933
4  d5bdf3446e86ce4ec67ce7a00f1cccc2    24.989403
Average L2 coefficients (top 12 by abs):
{
  "lgb:emb:openclip_vit_b16_224": 0.40855216339259615,
  "agg_min": 0.4067674784122034,
  "lgb:meta_stats": 0.24580635827245093,
  "lgb:emb:openclip_vit_b32_224": 0.22175726418799577,
  "lgb:emb:tf_efficientnetv2_s_in21k_224": 0.19633917222157776,
  "lgb:emb:vit_base_patch16_224_224": 0.17476867150676131,
  "lgb:emb:tf_efficientnet_b0_224": 0.16076190735151252,
  "lgb:emb:deit3_small_patch16_224_224": 0.15879837958488446,
  "lgb:emb:swin_small_patch4_window7_224_224": 0.14425655602618073,
  "agg_std": 0.13294185891827248,
  "lgb:hgb_meta_stats": -0.09074945078720045,
  "hgb:meta_stats": -0.09074

In [28]:
# PCA sweep for top embeddings (per-fold L2+PCA) with LightGBM; overwrite oof_/test_pred_ if improved
import os, time, glob, json
import numpy as np, pandas as pd
from sklearn.decomposition import PCA

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

try:
    import lightgbm as lgb
    lgbm_ok = True
except Exception as e:
    print('LightGBM not available for PCA sweep:', e)
    lgbm_ok = False

if lgbm_ok:
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    folds_df = pd.read_csv('folds.csv')
    id2fold = dict(zip(folds_df['Id'], folds_df['fold']))
    train_df['fold'] = train_df['Id'].map(id2fold)
    y = train_df['Pawpularity'].values.astype(float)

    def l2norm(a):
        n = np.linalg.norm(a, axis=1, keepdims=True) + 1e-12
        return a / n

    def load_fs(tag):
        X_tr = np.load(f'X_img_train_{tag}.npy')
        X_te = np.load(f'X_img_test_{tag}.npy')
        return X_tr, X_te

    # Targets to sweep: ViT/CLIP: 256/384/512; ConvNet: 384/512/640
    sweep_plan = {
        'openclip_vit_b32_224': [256, 384, 512],
        'openclip_vit_b16_224': [256, 384, 512],
        'swin_small_patch4_window7_224_224': [256, 384, 512],
        'tf_efficientnet_b0_224': [384, 512, 640],
        'vit_base_patch16_224_224': [256, 384, 512],
    }

    improved = {}
    for tag, ncomps in sweep_plan.items():
        tr_path = f'X_img_train_{tag}.npy'; te_path = f'X_img_test_{tag}.npy'
        if not (os.path.exists(tr_path) and os.path.exists(te_path)):
            print(f'[Skip] Missing arrays for {tag}')
            continue
        X_tr, X_te = load_fs(tag)
        base_oof_path = f'oof_emb:{tag}.npy'
        base_rmse = None
        if os.path.exists(base_oof_path):
            base_rmse = rmse(y, np.load(base_oof_path))
        else:
            print(f'[Info] No existing OOF for emb:{tag}; will treat best found as baseline')

        best = {'rmse': float('inf'), 'oof': None, 'test': None, 'n_comp': None}
        print(f'=== PCA sweep for emb:{tag} | candidates: {ncomps} ===', flush=True)
        for nc in ncomps:
            oof = np.zeros(len(train_df), dtype=float)
            te_pred_accum = np.zeros(len(test_df), dtype=float)
            t0 = time.time()
            for fold in range(5):
                tr_idx = np.where(train_df['fold'].values != fold)[0]
                va_idx = np.where(train_df['fold'].values == fold)[0]
                X_tr_fold = l2norm(X_tr[tr_idx])
                X_va_fold = l2norm(X_tr[va_idx])
                X_te_fold = l2norm(X_te)
                pca = PCA(n_components=min(nc, X_tr.shape[1]), whiten=False, random_state=42)
                X_tr_p = pca.fit_transform(X_tr_fold)
                X_va_p = pca.transform(X_va_fold)
                X_te_p = pca.transform(X_te_fold)
                dtrain = lgb.Dataset(X_tr_p, label=y[tr_idx])
                dvalid = lgb.Dataset(X_va_p, label=y[va_idx])
                params = dict(objective='regression', metric='rmse', learning_rate=0.03,
                              num_leaves=64, min_data_in_leaf=40, feature_fraction=0.8,
                              bagging_fraction=0.8, bagging_freq=1, lambda_l2=1.0, verbosity=-1)
                gbm = lgb.train(params, dtrain, num_boost_round=10000, valid_sets=[dvalid],
                                valid_names=['valid'], callbacks=[lgb.early_stopping(300), lgb.log_evaluation(200)])
                va_pred = np.clip(gbm.predict(X_va_p, num_iteration=gbm.best_iteration), 1.0, 100.0)
                te_pred = np.clip(gbm.predict(X_te_p, num_iteration=gbm.best_iteration), 1.0, 100.0)
                oof[va_idx] = va_pred
                te_pred_accum += te_pred
                print(f'  n_comp={nc} fold {fold} done | elapsed {time.time()-t0:.1f}s', flush=True)
            fs_rmse = rmse(y, oof)
            print(f'  -> n_comp={nc} OOF RMSE: {fs_rmse:.5f}', flush=True)
            if fs_rmse < best['rmse']:
                best = {'rmse': fs_rmse, 'oof': oof.copy(), 'test': (te_pred_accum/5.0).copy(), 'n_comp': nc}

        print(f'Best for emb:{tag}: n_comp={best["n_comp"]} | OOF RMSE: {best["rmse"]:.5f} | prev: {base_rmse}', flush=True)
        # Overwrite only if improvement vs existing (or if no baseline)
        if (base_rmse is None) or (best['rmse'] + 1e-6 < base_rmse):
            np.save(f'oof_emb:{tag}.npy', best['oof'])
            np.save(f'test_pred_emb:{tag}.npy', best['test'])
            improved[f'emb:{tag}'] = {'oof_rmse': best['rmse'], 'n_comp': best['n_comp']}
            print(f'  Saved improved OOF/test for emb:{tag}')
        else:
            print(f'  No improvement for emb:{tag}; kept existing files')

    print('PCA sweep improvements:', json.dumps(improved, indent=2))
else:
    print('Skipping PCA sweep: LightGBM unavailable.')

=== PCA sweep for emb:openclip_vit_b32_224 | candidates: [256, 384, 512] ===


Training until validation scores don't improve for 300 rounds


[200]	valid's rmse: 17.7075


[400]	valid's rmse: 17.7214


Early stopping, best iteration is:
[250]	valid's rmse: 17.6792
  n_comp=256 fold 0 done | elapsed 3.4s


...

[600]	valid's rmse: 18.4178


Early stopping, best iteration is:
[449]	valid's rmse: 18.4158
  n_comp=512 fold 4 done | elapsed 52.9s


  -> n_comp=512 OOF RMSE: 18.24591


Best for emb:vit_base_patch16_224_224: n_comp=256 | OOF RMSE: 18.19758 | prev: 18.197577009031072


  No improvement for emb:vit_base_patch16_224_224; kept existing files
PCA sweep improvements: {
  "emb:openclip_vit_b16_224": {
    "oof_rmse": 17.992867001924974,
    "n_comp": 256
  }
}


In [25]:
# Level-2 LightGBM stacker (nested CV) over LGBM + HGBR OOFs
import glob, json, time
import numpy as np, pandas as pd

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

try:
    import lightgbm as lgb
    lgbm_ok = True
except Exception as e:
    print('LightGBM not available for L2 stacker:', e)
    lgbm_ok = False

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
folds_df = pd.read_csv('folds.csv')
id2fold = dict(zip(folds_df['Id'], folds_df['fold']))
train_df['fold'] = train_df['Id'].map(id2fold)
y = train_df['Pawpularity'].values.astype(float)

# Load base OOF/Test matrices
oof_lgb_files = sorted(glob.glob('oof_*.npy'))
oof_hgb_files = sorted(glob.glob('oof_hgb_*.npy'))
def name_from(path, prefix):
    return path[len(prefix):-4]
names_lgb = [name_from(p, 'oof_') for p in oof_lgb_files]
names_hgb = [name_from(p, 'oof_hgb_') for p in oof_hgb_files]
cols = []
O_list, T_list = [], []
for p, n in zip(oof_lgb_files, names_lgb):
    O_list.append(np.load(p))
    T_list.append(np.load(f'test_pred_{n}.npy'))
    cols.append(f'lgb:{n}')
for p, n in zip(oof_hgb_files, names_hgb):
    O_list.append(np.load(p))
    T_list.append(np.load(f'test_pred_hgb_{n}.npy'))
    cols.append(f'hgb:{n}')
if len(O_list) == 0:
    raise SystemExit('No base models found for L2 stacker')
O = np.vstack(O_list).T
T = np.vstack(T_list).T
print('L2 LightGBM: base features:', len(cols), '| shapes:', O.shape, T.shape)

# Row-wise aggregation features
def add_row_aggs(M):
    mean = M.mean(axis=1, keepdims=True)
    std = M.std(axis=1, keepdims=True)
    vmin = M.min(axis=1, keepdims=True)
    vmax = M.max(axis=1, keepdims=True)
    return np.hstack([M, mean, std, vmin, vmax])
O_ext = add_row_aggs(O)
T_ext = add_row_aggs(T)

if lgbm_ok:
    meta_oof = np.zeros(len(train_df), dtype=float)
    meta_test_accum = np.zeros(len(test_df), dtype=float)
    t0 = time.time()
    for fold in range(5):
        tr_idx = np.where(train_df['fold'].values != fold)[0]
        va_idx = np.where(train_df['fold'].values == fold)[0]
        X_tr, X_va = O_ext[tr_idx], O_ext[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]
        dtrain = lgb.Dataset(X_tr, label=y_tr)
        dvalid = lgb.Dataset(X_va, label=y_va)
        params = dict(
            objective='regression', metric='rmse', learning_rate=0.03,
            num_leaves=31, min_data_in_leaf=60, feature_fraction=0.8,
            bagging_fraction=0.8, bagging_freq=1, lambda_l2=2.0, verbosity=-1
        )
        gbm = lgb.train(params, dtrain, num_boost_round=20000, valid_sets=[dvalid], valid_names=['valid'],
                        callbacks=[lgb.early_stopping(150), lgb.log_evaluation(200)])
        va_pred = gbm.predict(X_va, num_iteration=gbm.best_iteration)
        meta_oof[va_idx] = va_pred
        meta_test_accum += gbm.predict(T_ext, num_iteration=gbm.best_iteration)
        print(f'L2 LGBM fold {fold} RMSE: {rmse(y_va, va_pred):.5f} | iters: {gbm.best_iteration} | elapsed {time.time()-t0:.1f}s', flush=True)

    meta_rmse = rmse(y, meta_oof)
    meta_test = meta_test_accum / 5.0
    print(f'L2 LGBM meta OOF RMSE: {meta_rmse:.5f}')
    np.save('oof_L2_lgbm.npy', meta_oof)
    np.save('test_pred_L2_lgbm.npy', meta_test)
    sub = pd.DataFrame({'Id': test_df['Id'], 'Pawpularity': np.clip(meta_test, 1.0, 100.0)})
    sub.to_csv('submission.csv', index=False)
    print('Saved submission.csv (L2 LGBM). Head:')
    print(sub.head())
else:
    print('Skipping L2 LightGBM: LightGBM not available')

L2 LightGBM: base features: 28 | shapes: (8920, 28) (992, 28)
Training until validation scores don't improve for 150 rounds
[200]	valid's rmse: 17.3565


Early stopping, best iteration is:
[90]	valid's rmse: 17.199
L2 LGBM fold 0 RMSE: 17.19895 | iters: 90 | elapsed 0.2s


Training until validation scores don't improve for 150 rounds
[200]	valid's rmse: 18.0293


Early stopping, best iteration is:
[66]	valid's rmse: 17.9094
L2 LGBM fold 1 RMSE: 17.90943 | iters: 66 | elapsed 0.4s


Training until validation scores don't improve for 150 rounds
[200]	valid's rmse: 18.1887


...

Early stopping, best iteration is:
[99]	valid's rmse: 17.5192
L2 LGBM fold 3 RMSE: 17.51919 | iters: 99 | elapsed 0.9s


Training until validation scores don't improve for 150 rounds
[200]	valid's rmse: 18.157


Early stopping, best iteration is:
[75]	valid's rmse: 17.963


L2 LGBM fold 4 RMSE: 17.96302 | iters: 75 | elapsed 1.1s


L2 LGBM meta OOF RMSE: 17.72981
Saved submission.csv (L2 LGBM). Head:
                                 Id  Pawpularity
0  ee51b99832f1ba868f646df93d2b6b81    61.325353
1  caddfb3f8bff9c4b95dbe022018eea21    35.590551
2  582eeabd4a448a53ebb79995888a4b0b    33.021473
3  afc1ad7f0c5eea880759d09e77f7deee    30.657687
4  d5bdf3446e86ce4ec67ce7a00f1cccc2    28.041714


In [26]:
# Enrich meta features (cheap image stats) and retrain meta models (LGBM + HGBR); overwrite OOF/test for meta_stats
import os, time, json, math
import numpy as np, pandas as pd, cv2
from sklearn.ensemble import HistGradientBoostingRegressor

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def colorfulness_bgr(img):
    # img BGR uint8
    B, G, R = cv2.split(img.astype(np.float32))
    rg = R - G
    yb = 0.5 * (R + G) - B
    std_rg, mean_rg = np.std(rg), np.mean(rg)
    std_yb, mean_yb = np.std(yb), np.mean(yb)
    return float(np.sqrt(std_rg**2 + std_yb**2) + 0.3 * np.sqrt(mean_rg**2 + mean_yb**2))

def entropy_channel(ch):
    # ch uint8 0..255
    hist = cv2.calcHist([ch], [0], None, [256], [0,256]).ravel()
    p = hist / (np.sum(hist) + 1e-12)
    p = p[p > 0]
    return float(-np.sum(p * np.log2(p)))

def center_brightness_ratio(gray):
    h, w = gray.shape
    ch0, ch1 = int(h*0.25), int(h*0.75)
    cw0, cw1 = int(w*0.25), int(w*0.75)
    center = gray[ch0:ch1, cw0:cw1]
    g_mean = float(np.mean(gray)) + 1e-6
    c_mean = float(np.mean(center))
    return float(c_mean / g_mean)

def otsu_foreground_fraction(gray):
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return float(np.mean(th > 0))

def compute_features_row(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        return None
    h, w = img.shape[:2]
    aspect = (w / max(h,1.0)) if h>0 else 0.0
    # original stats
    bgr_means = img.reshape(-1,3).mean(axis=0).tolist()
    bgr_stds = img.reshape(-1,3).std(axis=0).tolist()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lap_var = float(cv2.Laplacian(gray, cv2.CV_64F).var())
    # enrichments
    gray_mean = float(np.mean(gray))
    gray_std = float(np.std(gray))
    p10 = float(np.percentile(gray, 10))
    p50 = float(np.percentile(gray, 50))
    p90 = float(np.percentile(gray, 90))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    hsv_means = [float(np.mean(H)), float(np.mean(S)), float(np.mean(V))]
    hsv_stds = [float(np.std(H)), float(np.std(S)), float(np.std(V))]
    ent_s = entropy_channel(S)
    ent_v = entropy_channel(V)
    edges = cv2.Canny(gray, 100, 200)
    edge_density = float(np.mean(edges > 0))
    colorful = colorfulness_bgr(img)
    center_ratio = center_brightness_ratio(gray)
    otsu_frac = otsu_foreground_fraction(gray)
    feats = [w, h, aspect] + bgr_means + bgr_stds + [lap_var,
             gray_mean, gray_std, p10, p50, p90] + hsv_means + hsv_stds + [ent_s, ent_v,
             edge_density, colorful, center_ratio, otsu_frac]
    return np.array(feats, dtype=np.float32)

meta_cols = ['Subject Focus','Eyes','Face','Near','Action','Accessory','Group','Collage','Human','Occlusion','Info','Blur']
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
y = train_df['Pawpularity'].values.astype(float)

def build_meta_matrix(df, folder, log_every=1000):
    Xm = df[meta_cols].astype(np.float32).values
    feats = []
    ids = df['Id'].tolist()
    t0 = time.time()
    for i, id_ in enumerate(ids):
        f = compute_features_row(os.path.join(folder, f'{id_}.jpg'))
        if f is None:
            f = np.zeros(3+6+1+5+6+2+1+1+1, dtype=np.float32)  # fallback to zeros for enrichments+orig stats
        feats.append(f)
        if (i+1) % log_every == 0:
            print(f'  {folder}: {i+1}/{len(ids)} | elapsed {time.time()-t0:.1f}s', flush=True)
    Xf = np.vstack(feats)
    X = np.concatenate([Xm, Xf], axis=1)
    return X

print('Building enriched meta features...')
t0_all = time.time()
X_tr = build_meta_matrix(train_df, 'train')
X_te = build_meta_matrix(test_df, 'test')
np.save('X_meta_stats_train.npy', X_tr)
np.save('X_meta_stats_test.npy', X_te)
print('Saved enriched X_meta_stats_train.npy', X_tr.shape, '| X_meta_stats_test.npy', X_te.shape, '| time', f'{time.time()-t0_all:.1f}s')

# Retrain meta_stats models: LightGBM and HGBR
folds_df = pd.read_csv('folds.csv')
train_df = train_df.merge(folds_df, on='Id', how='left')

# Train LGBM for meta_stats
try:
    import lightgbm as lgb
    lgb_ok = True
except Exception as e:
    print('LightGBM not available for meta retrain:', e); lgb_ok = False

if lgb_ok:
    oof = np.zeros(len(train_df), dtype=float)
    te_acc = np.zeros(len(test_df), dtype=float)
    for fold in range(5):
        tr_idx = np.where(train_df['fold'].values != fold)[0]
        va_idx = np.where(train_df['fold'].values == fold)[0]
        dtrain = lgb.Dataset(X_tr[tr_idx], label=y[tr_idx])
        dvalid = lgb.Dataset(X_tr[va_idx], label=y[va_idx])
        params = dict(objective='regression', metric='rmse', learning_rate=0.03,
                      num_leaves=64, min_data_in_leaf=40, feature_fraction=0.8,
                      bagging_fraction=0.8, bagging_freq=1, lambda_l2=1.0, verbosity=-1)
        gbm = lgb.train(params, dtrain, num_boost_round=10000, valid_sets=[dvalid],
                        valid_names=['valid'], callbacks=[lgb.early_stopping(300), lgb.log_evaluation(200)])
        va_pred = gbm.predict(X_tr[va_idx], num_iteration=gbm.best_iteration)
        te_pred = gbm.predict(X_te, num_iteration=gbm.best_iteration)
        oof[va_idx] = np.clip(va_pred, 1.0, 100.0)
        te_acc += np.clip(te_pred, 1.0, 100.0)
        print(f'  meta_stats [LGBM] fold {fold} RMSE: {rmse(y[va_idx], oof[va_idx]):.4f}', flush=True)
    te_mean = te_acc / 5.0
    print('meta_stats [LGBM] OOF RMSE:', f'{rmse(y, oof):.5f}')
    np.save('oof_meta_stats.npy', oof)
    np.save('test_pred_meta_stats.npy', te_mean)

# Train HGBR for meta_stats
oof_h = np.zeros(len(train_df), dtype=float)
te_acc_h = np.zeros(len(test_df), dtype=float)
hgbr = HistGradientBoostingRegressor(learning_rate=0.05, max_iter=2500, max_leaf_nodes=31,
                                     min_samples_leaf=30, l2_regularization=1.0,
                                     validation_fraction=0.12, early_stopping=True, random_state=42)
for fold in range(5):
    tr_idx = np.where(train_df['fold'].values != fold)[0]
    va_idx = np.where(train_df['fold'].values == fold)[0]
    hgbr.fit(X_tr[tr_idx], y[tr_idx])
    va_pred = np.clip(hgbr.predict(X_tr[va_idx]), 1.0, 100.0)
    te_pred = np.clip(hgbr.predict(X_te), 1.0, 100.0)
    oof_h[va_idx] = va_pred
    te_acc_h += te_pred
    print(f'  meta_stats [HGBR] fold {fold} RMSE: {rmse(y[va_idx], va_pred):.4f}', flush=True)
te_mean_h = te_acc_h / 5.0
print('meta_stats [HGBR] OOF RMSE:', f'{rmse(y, oof_h):.5f}')
np.save('oof_hgb_meta_stats.npy', oof_h)
np.save('test_pred_hgb_meta_stats.npy', te_mean_h)
print('Meta retrain complete.')

Building enriched meta features...


  train: 1000/8920 | elapsed 95.9s


  train: 2000/8920 | elapsed 183.3s


  train: 3000/8920 | elapsed 277.4s


  train: 4000/8920 | elapsed 376.7s


...

  meta_stats [HGBR] fold 1 RMSE: 20.6704


  meta_stats [HGBR] fold 2 RMSE: 20.7867


  meta_stats [HGBR] fold 3 RMSE: 20.6797


  meta_stats [HGBR] fold 4 RMSE: 20.6813


meta_stats [HGBR] OOF RMSE: 20.65130
Meta retrain complete.


In [27]:
# Extract extra embeddings: OpenCLIP ViT-B/16 and BEiT-Base (in22k) @224 on CPU
import os, time, gc, math, subprocess, sys
import numpy as np, pandas as pd
import torch
import timm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from timm.data import resolve_data_config, create_transform

torch.set_num_threads(8)

def ensure_openclip():
    try:
        import open_clip  # noqa
    except Exception:
        print("Installing open-clip-torch (no-deps)...", flush=True)
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-deps', '--upgrade-strategy', 'only-if-needed', 'open-clip-torch'], check=True)
    for pkg in ['ftfy', 'regex']:
        try:
            __import__(pkg)
        except Exception:
            print(f"Installing {pkg}...", flush=True)
            subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade-strategy', 'only-if-needed', pkg], check=True)
    return True

class ImageDataset(Dataset):
    def __init__(self, ids, folder, transform):
        self.ids = ids
        self.folder = folder
        self.transform = transform
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        img_id = self.ids[idx]
        path = os.path.join(self.folder, f"{img_id}.jpg")
        img = Image.open(path).convert('RGB')
        img = self.transform(img)
        return img

def extract_timm_embeddings(model_name, img_size=224, batch_size=128, num_workers=8):
    tr_out = f"X_img_train_{model_name.replace('/', '_')}_{img_size}.npy"
    te_out = f"X_img_test_{model_name.replace('/', '_')}_{img_size}.npy"
    if os.path.exists(tr_out) and os.path.exists(te_out):
        print(f"[Skip] {model_name} exists: {tr_out}, {te_out}")
        return
    t0 = time.time()
    print(f"\n[Emb] {model_name} @ {img_size}", flush=True)
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    train_ids = train_df['Id'].tolist()
    test_ids = test_df['Id'].tolist()

    model = timm.create_model(model_name, pretrained=True, num_classes=0, global_pool='avg')
    model.eval().to('cpu')

    cfg = resolve_data_config({}, model=model)
    cfg['input_size'] = (3, img_size, img_size)
    transform = create_transform(**cfg, is_training=False)

    train_ds = ImageDataset(train_ids, 'train', transform)
    test_ds = ImageDataset(test_ids, 'test', transform)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=False)

    with torch.no_grad():
        sample = next(iter(train_loader))[:1]
        emb_dim = model(sample).shape[1]
    print(f"Embedding dim: {emb_dim}")

    def run_loader(loader, n_items):
        X = np.zeros((n_items, emb_dim), dtype=np.float32)
        i0 = 0
        with torch.no_grad():
            for i, xb in enumerate(loader):
                feats = model(xb).cpu().numpy().astype(np.float32)
                X[i0:i0+feats.shape[0]] = feats
                i0 += feats.shape[0]
                if (i+1) % 20 == 0:
                    print(f"  Batches {i+1}/{math.ceil(n_items/loader.batch_size)} | rows {i0}/{n_items} | elapsed {time.time()-t0:.1f}s", flush=True)
        return X

    X_tr = run_loader(train_loader, len(train_ds))
    X_te = run_loader(test_loader, len(test_ds))
    np.save(tr_out, X_tr); np.save(te_out, X_te)
    print(f"Saved {tr_out} {X_tr.shape}, {te_out} {X_te.shape} | time {time.time()-t0:.1f}s")
    del model, X_tr, X_te, train_loader, test_loader, train_ds, test_ds
    gc.collect()

def extract_openclip_vitb16(img_size=224, batch_size=128, num_workers=8):
    tag = f"openclip_vit_b16_{img_size}"
    tr_out = f"X_img_train_{tag}.npy"
    te_out = f"X_img_test_{tag}.npy"
    if os.path.exists(tr_out) and os.path.exists(te_out):
        print(f"[Skip] OpenCLIP ViT-B/16 exists: {tr_out}, {te_out}")
        return
    ensure_openclip()
    import open_clip
    from torchvision import transforms as T
    t0 = time.time()
    print(f"\n[Emb] OpenCLIP ViT-B/16 @ {img_size}", flush=True)
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='laion2b_s34b_b88k', device='cpu')
    model.eval()
    preprocess = T.Compose([
        T.Resize(img_size, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(img_size),
        T.ToTensor(),
        T.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711)),
    ])

    class OpenClipDS(Dataset):
        def __init__(self, ids, folder):
            self.ids = ids; self.folder = folder
        def __len__(self): return len(self.ids)
        def __getitem__(self, idx):
            img = Image.open(os.path.join(self.folder, f"{self.ids[idx]}.jpg")).convert('RGB')
            return preprocess(img)

    train_df = pd.read_csv('train.csv'); test_df = pd.read_csv('test.csv')
    train_ids = train_df['Id'].tolist(); test_ids = test_df['Id'].tolist()
    train_loader = DataLoader(OpenClipDS(train_ids, 'train'), batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(OpenClipDS(test_ids, 'test'), batch_size=batch_size, shuffle=False, num_workers=num_workers)

    with torch.no_grad():
        sample = next(iter(train_loader))[:1]
        emb_dim = model.encode_image(sample).shape[1]
    print(f"Embedding dim: {emb_dim}")

    def run_loader(loader, n_items):
        X = np.zeros((n_items, emb_dim), dtype=np.float32)
        i0 = 0
        with torch.no_grad():
            for i, xb in enumerate(loader):
                feats = model.encode_image(xb).float().cpu().numpy().astype(np.float32)
                X[i0:i0+feats.shape[0]] = feats
                i0 += feats.shape[0]
                if (i+1) % 20 == 0:
                    print(f"  Batches {i+1}/{math.ceil(n_items/loader.batch_size)} | rows {i0}/{n_items} | elapsed {time.time()-t0:.1f}s", flush=True)
        return X

    X_tr = run_loader(train_loader, len(train_ids))
    X_te = run_loader(test_loader, len(test_ids))
    np.save(tr_out, X_tr); np.save(te_out, X_te)
    print(f"Saved {tr_out} {X_tr.shape}, {te_out} {X_te.shape} | time {time.time()-t0:.1f}s")
    del model, X_tr, X_te, train_loader, test_loader
    gc.collect()

# Execute extractions
try:
    extract_openclip_vitb16(img_size=224, batch_size=128, num_workers=8)
except Exception as e:
    print(f"[Warn] OpenCLIP ViT-B/16 extraction failed: {e}")

try:
    extract_timm_embeddings('beit_base_patch16_224_in22k', img_size=224, batch_size=128, num_workers=8)
except Exception as e:
    print(f"[Warn] BEiT-Base extraction failed: {e}")

print('Extra embeddings extraction complete.')


[Emb] OpenCLIP ViT-B/16 @ 224


  Batches 20/70 | rows 2560/8920 | elapsed 127.7s


  Batches 40/70 | rows 5120/8920 | elapsed 248.6s


  Batches 60/70 | rows 7680/8920 | elapsed 370.1s


Saved X_img_train_openclip_vit_b16_224.npy (8920, 512), X_img_test_openclip_vit_b16_224.npy (992, 512) | time 477.3s



[Emb] beit_base_patch16_224_in22k @ 224


[Warn] BEiT-Base extraction failed: Unknown model (beit_base_patch16_224_in22k)
Extra embeddings extraction complete.
